In [1]:
# Importing necessary libraries
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .envrc for the Chat GPT
load_dotenv(".envrc")

True

### Load and Prepare the data

To prepare the data to be ingested in the elastic search we will:
1. Load the cleaned data 
2. Convert the cleaned data to a list of dictionaries - what elastic search expects as a document

In [3]:
# Read the prepared data
data = pd.read_csv('data/investment_data.csv')
# Explore the first rows of data
data.head()

,question,answer,context,ticker,filing,company,id
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA,2023_10K,Nvidia Corporation,4f2ccc3b
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA,2023_10K,Nvidia Corporation,ee4ed04f
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA,2023_10K,Nvidia Corporation,7eac6b57
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA,2023_10K,Nvidia Corporation,eb49bbd0
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA,2023_10K,Nvidia Corporation,3e4c199c


In [4]:
# Convert the dataframe into a list of dictionaries for each record
records = data.to_dict(orient='records')
# Examine the first 2 QA records
records[:2]

[{'question': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
  'answer': 'NVIDIA initially focused on PC graphics.',
  'context': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
  'ticker': 'NVDA',
  'filing': '2023_10K',
  'company': 'Nvidia Corporation',
  'id': '4f2ccc3b'},
 {'question': 'What are some of the recent applications of GPU-powered deep learning as mentioned by NVIDIA?',
  'answer': 'Recent applications of GPU-powered deep learning include recommendation systems, large language models, and generative AI.',
  'context': 'Some of the most recent applications of GPU-powered deep learning include recommendation systems, which are AI algorithms trained to understand the preferences, previous decisions, and characteristics of people and products using data gathered about their interactions, large language models, which can recognize, summa

### Set up Elastic Search

In this part we will configure and create an Elastic search index to store the QA information. To do that we will:
1. Run the elastic search docker container and bind it to a local port
2. Import and initialize the elastic search client on that port
4. Configure the schema and name of the index - will contain both text data along with embedding data - for now it will be be just the text fields
5. Create the elastic search index
6. Add all the documents to the index

In [5]:
# Initialize the client 
es_client = Elasticsearch('http://localhost:9200')

In [6]:
# Create the Schema of the Elastic Search Index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "context": {"type": "text"},
            "ticker": {"type": "keyword"}, 
            "company": {"type": "keyword"},
            "id": {"type": "keyword"}
        }
    }
}

In [7]:
# Provide the name of the index
index_name = "investment-info"
# Check if the index exists
if es_client.indices.exists(index=index_name):
    # Delete the existing index
    es_client.indices.delete(index=index_name)
# Create the elastic search index
response = es_client.indices.create(index=index_name, body=index_settings)
# Verify that elastic search is created
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'investment-info'}


In [8]:
# Fetch all the documents into the elastic search index
for record in tqdm(records):
    es_client.index(index = index_name, document=record)

100%|██████████| 6990/6990 [00:14<00:00, 483.22it/s]


### Information Retrieval

To retrieve the relevant records to user's query we will:
1. Create the request body for the elastic search by specified the fields need to examined for the retrieval process
2. Make a search request to the created index
3. Parse the documents retrieved from the search request  

In [9]:
# Create a text search function to retrieve document form the elastic search
def document_search(query):
    # Create the query
    search_query = {
        # Specifying the number of documents to be retrieved
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        # Add the user query
                        "query": query,
                        # Include the text fields to search
                        "fields": ["question^2", "answer", "context"], # Give a boosting of 2 in the question field
                        "type": "best_fields"
                    }
            }
        }
    }
    }
    # Query the Elastic Search 
    response = es_client.search(index=index_name, body=search_query)

    # Parse the response of elastic search
    results = []
    for hit in response['hits']['hits']:
        results.append(hit['_source'])
    
    return results

In [11]:
# Test the retrieval part
query = "Where can I find NVIDIA's financial reports?"
retrieved_docs = document_search(query)

retrieved_docs[:2]

[{'question': "Where can NVIDIA's financial reports be accessed?",
  'answer': "NVIDIA's financial reports can be accessed for free on their website or through links provided on the website as soon as they are electronically filed or furnished to the SEC.",
  'context': 'Available Information Our annual reports on Form 10-K, quarterly reports on Form 10-Q, current reports on Form 8-K and, if applicable, amendments to those reports filed or furnished pursuant to Section 13(a) or 15(d) of the Securities Exchange Act of 1934, as amended, or the Exchange Act, are available free of charge on or through our website, http://www.nvidia.com, as soon as reasonably practicable after we electronically file such material with, or furnish it to, the Securities and Exchange Commission, or the SEC.',
  'ticker': 'NVDA',
  'filing': '2023_10K',
  'company': 'Nvidia Corporation',
  'id': 'cf09a223'},
 {'question': "Where can one find information about a company's legal proceedings in its financial repor

### Prompt Generation

To generate a prompt for a model using information from our own knowledge base we need to:
1. Create a prompt template that will have some instuction for the model to follow - role and task
2. We will create the context for the model to answer upon using the most relevant documents retrieved from our knowledge base
3. Include the query of the user along with the relevant context in a prompt for the LLM to utilize to provide an informative answer

In [12]:
# Create the function will generate the context
def context_generation(retrieved_docs):
    # Initialize context
    context = ''
    # Create context for each document
    for record in retrieved_docs:
        context += 'question: {question} answer: {answer} \n\n'.format(**record)
    # Return context
    return context

In [13]:
# Create a function to create the prompt
def prompt_generation(query, retrieved_docs):
    # Create the prompt template
    prompt_template = '''
    You are an Investment Assistant with deep expertise in financial markets.
    Provide a fact-based answer to the user's QUESTION strictly using the information provided in the CONTEXT. 
    Do not include opinions or external knowledge.

    QUESTION: {question}

    CONTEXT: 
    {context}
    '''.strip()
    # Create the context
    prompt_context = {}
    prompt_context['question'] = query
    prompt_context['context'] = context_generation(retrieved_docs)
    # Generate the prompt
    prompt = prompt_template.format(**prompt_context)
    return prompt

### LLM set up

For the LLM part of the RAG system we will use OpenAI's API and more specifically the gpt-4o-mini model. To use this model in our application:
1. Went to the OpenAI and generated a api key
2. Saved and load the key to an environment variable
3. Initialize the OpenAI instance to make a request to the OpenAI API endpoint
4. Create a function to make a request to the completion model and parse its response

In [14]:
# Initialize the openai instance
client = OpenAI()

In [15]:
# Create the function to request an answer from an LLM
def ask_llm(prompt, model='gpt-4o-mini'):
    # Create the request body and make the request
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

### RAG System

To create a fully functional RAG system we will create a function:
1. To search the provided knowledge base and retieve the necessary documents
2. To create the prompt using the user query and the context retrieved from the search
3. Use an LLM model to generate a response based on the user query

In [17]:
# Create the RAG function combining all the above information
def investment_assistant(query):
    # Retrieve the relevant documents
    relevant_docs = document_search(query)
    # Create the prompt for the model
    prompt = prompt_generation(query, relevant_docs)
    # Ask the LLM model
    response = ask_llm(prompt)
    return response

In [18]:
# Create a prompt to test rag system
query = "Where can I find NVIDIA's financial reports?"
# Test RAG system
answer = investment_assistant(query)
# Examine the response
print(answer)

NVIDIA's financial reports can be accessed for free on their website or through links provided on the website as soon as they are electronically filed or furnished to the SEC.
